In [85]:
import pandas as pd
import numpy as np
import re

from collections import Counter
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
data = pd.read_csv('main_task.csv')

Описание столбцов
1. Restaurant_id — идентификационный номер ресторана;
2. City — город, в котором находится ресторан;
3. Cuisine Style — стиль или стили, к которым можно отнести блюда, предлагаемые в ресторане;
4. Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
5. Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
6. Price Range — диапазон цен в ресторане;
7. Number of Reviews — количество отзывов о ресторане;
8. Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
9. URL_TA — URL страницы ресторана на TripAdvosor;
10. ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [4]:
data.head(10)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
5,id_1418,Oporto,NaN,1419.0,3.0,NaN,2.0,"[['There are better 3 star hotel bars', 'Amazi...",/Restaurant_Review-g189180-d12503536-Reviews-D...,d12503536
6,id_1720,Milan,"['Italian', 'Pizza']",1722.0,4.0,$,50.0,"[['Excellent simple local eatery.', 'Excellent...",/Restaurant_Review-g187849-d5808504-Reviews-Pi...,d5808504
7,id_825,Bratislava,['Italian'],826.0,3.0,NaN,9.0,"[['Wasting of money', 'excellent cuisine'], ['...",/Restaurant_Review-g274924-d3199765-Reviews-Ri...,d3199765
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029
9,id_4209,Rome,"['Italian', 'Pizza', 'Fast Food']",4210.0,4.0,$,55.0,"[['Clean efficient staff', 'Nice little pizza ...",/Restaurant_Review-g187791-d8020681-Reviews-Qu...,d8020681


In [5]:
# посмотрим на заполнение стобцов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


9283 строки не содержат информацию о стиле/стилях кухни ресторанов (23,2% от количества объектов), 13886 строк не содержат информацию о диапазоне цен (34,7%), 2543 строки не содержат информации о количестве отзывов (6,3%), хотя тексты отзывов присутствуют у всех ресторанов. Посмотрим на последние данные 

In [12]:
reviews_without_number = data[data['Number of Reviews'].isnull()]
reviews_without_number.head(10)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029
21,id_5844,Madrid,NaN,5847.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d10058810-Reviews-B...,d10058810
32,id_1327,Budapest,NaN,1328.0,5.0,NaN,NaN,"[['Absolutely amazing, tasty, fresh, cheap Ch....",/Restaurant_Review-g274887-d8791087-Reviews-Bu...,d8791087
102,id_1409,Budapest,"['French', 'European', 'Hungarian']",1410.0,5.0,NaN,NaN,"[[], []]",/Restaurant_Review-g274887-d13197631-Reviews-L...,d13197631
108,id_2047,Prague,"['Italian', 'Mediterranean']",2050.0,5.0,$$ - $$$,NaN,"[[], []]",/Restaurant_Review-g274707-d12243659-Reviews-T...,d12243659
167,id_8305,Paris,NaN,8306.0,5.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187147-d13284434-Reviews-S...,d13284434
180,id_5023,Berlin,NaN,5025.0,4.0,NaN,NaN,"[['Free Wi-Fi, Peaceful, Pleasant'], ['01/19/2...",/Restaurant_Review-g187323-d12068247-Reviews-K...,d12068247
183,id_3566,Berlin,['Italian'],3568.0,5.0,NaN,NaN,"[['Wonderful ambiance, delicious food, fantas....",/Restaurant_Review-g187323-d2424319-Reviews-Fo...,d2424319
187,id_2847,Lisbon,NaN,2850.0,4.0,NaN,NaN,[['Affordable little cafe in the middle of Al....,/Restaurant_Review-g189158-d13139218-Reviews-C...,d13139218
199,id_16034,London,NaN,16046.0,2.0,NaN,NaN,"[[], []]",/Restaurant_Review-g186338-d7243246-Reviews-Yu...,d7243246


Оказывается, как видно, тексты отзывов есть не у всех ресторанов, но некоторые значения в столбце Reviews заполнены пустыми списками, что не равняется NaN. Можно будет заполнить отсутствующее значение в ячейке Number of Reviews для ресторанов, у которых есть список отзывов, взяв его длину. Для остальных заполнить значения нулями (как вариант, тут может измениться логика, пока не проходил дальше по заданиям)

Еще посмотрим на значения диапазона цен и рейтинга

In [27]:
print('Price range values = {}'.format(data['Price Range'].unique()))
print('Rating interval = {}'.format(data['Rating'].value_counts(bins=1).index[0]))
print('Rating values = {}'.format(data['Rating'].unique()))

Price range values = ['$$ - $$$' nan '$$$$' '$']
Rating interval = (0.995, 5.0]
Rating values = [ 3.5  4.   4.5  5.   3.   2.5  2.   1.   1.5]


##  Задание

<b>Из постановки задачи: мы должны использовать в модели датафрейм, содержащий только количественные признаки и не содержащий None-значений. На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типа object, и заполним пропущенные значения (None или NaN) каким-то одним значением (нулём или средним арифметическим) для  всего столбца.<b>

Таким образом, удалим столбцы типа object, убедимся, что количество отзывов и место ресторана - целые значения и заполним нолями пропуски в столбце количества отзывов. Сохраним все в отдельную переменную task_df 

In [43]:
# проверим, что количество отзывов и места - целые числа
number_of_reviews = data['Number of Reviews'].dropna().apply(lambda x: x.is_integer())
ranking = data['Ranking'].dropna().apply(lambda x: x.is_integer())

print(number_of_reviews.unique(), ranking.unique())

[ True] [ True]


In [9]:
task_df = data.drop(labels=['Restaurant_id','City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'], axis=1).fillna(0)

In [24]:
# Убедимся, что не осталось пропущенных значений
values = task_df.isnull().values
values[values == True].size

0

In [25]:
# Итоговый датафрейм
task_df.head()

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0


Применим базовую модель из ноутбука you_first_model

In [37]:
X = task_df.iloc[:, [0, 2]]
y = task_df.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [38]:
regr = RandomForestRegressor(n_estimators=100)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.4316716077380952


Метрика MAE приняла значение 0.43. Учитывая, что рейтинг изменяется в интервале от 1 до 5 с шагом 0.5 (Rating values = [ 3.5  4.   4.5  5.   3.   2.5  2.   1.   1.5]), отклонение получилось довольно существенным. Очевидно, что нужно вводить больше признаков. Так как признак количества отзывов не заполнен для 2543-х объектов, можно попробовать предзаполнить его там, где есть список отзывов, взяв длину этого списка

### Функции для обработки данных 

In [3]:
# Преобразование строкового представления списка в list (для признака "стили кухни")
def cuisines_to_list(string):
    return string.strip('[]').split(', ')

In [115]:
# Преобразование строкового представления списков в list (для признака Reviews)
def reviews_to_list(string, dates=False):
    pattern = re.compile('\'[^\']+\'')
    reviews = [word[1:-1] for word in pattern.findall(string)]
    
    if reviews == []:
        return [np.nan] * 2
    
    if dates:
        return reviews[-2:] if len(reviews) == 4 else [reviews[-1], np.nan]
    else:
        return reviews[:2] if len(reviews) == 4 else [reviews[0], np.nan]

## Наполнение выборки данными 

In [12]:
# Сколько городов представлено в наборе данных?
data['City'].nunique()

31

In [8]:
data['City'].unique()

array(['Paris', 'Stockholm', 'London', 'Berlin', 'Munich', 'Oporto',
       'Milan', 'Bratislava', 'Vienna', 'Rome', 'Barcelona', 'Madrid',
       'Dublin', 'Brussels', 'Zurich', 'Warsaw', 'Budapest', 'Copenhagen',
       'Amsterdam', 'Lyon', 'Hamburg', 'Lisbon', 'Prague', 'Oslo',
       'Helsinki', 'Edinburgh', 'Geneva', 'Ljubljana', 'Athens',
       'Luxembourg', 'Krakow'], dtype=object)

In [11]:
# Сколько ресторанов расположены в столичных городах?
data.query('City not in ["Munich", "Oporto", "Milan", "Barcelona", "Zurich", "Lyon", "Hamburg", "Geneva", "Krakow"]').shape[0]

30424

In [5]:
# Сколько типов кухонь представлено в наборе данных?
all_cuisines = set()

for cuisine in data['Cuisine Style'].dropna():
    all_cuisines.update(cuisine.strip('[]').split(', '))
    
len(all_cuisines)

125

In [33]:
# Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.
c = Counter()

for cuisines in data['Cuisine Style'].dropna():
    for cuisine in cuisines.strip('[]').split(', '):
        c[cuisine.strip("\'")] += 1
        
print(c)

Counter({'Vegetarian Friendly': 11189, 'European': 10060, 'Mediterranean': 6277, 'Italian': 5964, 'Vegan Options': 4486, 'Gluten Free Options': 4113, 'Bar': 3297, 'French': 3190, 'Asian': 3011, 'Pizza': 2849, 'Spanish': 2798, 'Pub': 2449, 'Cafe': 2325, 'Fast Food': 1705, 'British': 1595, 'International': 1584, 'Seafood': 1505, 'Japanese': 1464, 'Central European': 1393, 'American': 1315, 'Sushi': 1156, 'Chinese': 1145, 'Portuguese': 1107, 'Indian': 1041, 'Middle Eastern': 782, 'Thai': 743, 'Wine Bar': 697, 'German': 662, 'Healthy': 620, 'Greek': 604, 'Halal': 598, 'Czech': 595, 'Fusion': 577, 'Steakhouse': 573, 'Barbecue': 555, 'Contemporary': 523, 'Vietnamese': 513, 'Eastern European': 496, 'Soups': 494, 'Grill': 490, 'Gastropub': 471, 'Mexican': 445, 'Turkish': 444, 'Delicatessen': 392, 'Austrian': 380, 'South American': 372, 'Polish': 365, 'Hungarian': 352, 'Scandinavian': 342, 'Lebanese': 329, 'Latin': 302, 'Diner': 295, 'Dutch': 294, 'Irish': 284, 'Belgian': 270, 'Street Food': 26

In [43]:
# Какое среднее количество кухонь предлагается в одном ресторане?
# Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. 
# Ответ округлите до одного знака после запятой.

cuisines = data.loc[:, 'Cuisine Style'].fillna('[Vegetarian Friendly]')
cuisines = cuisines.apply(lambda x: cuisines_to_list(x))

cuisines_in_all_restaurants = []

for cuisine in cuisines:
    cuisines_in_all_restaurants.append(len(cuisine))

print(np.mean(cuisines_in_all_restaurants))

2.6224


In [60]:
#datetime('09/22/2017')
pd.to_datetime('09/22/2017').strftime('%Y-%m-%d')

'2017-09-22'

In [116]:
reviews = data.loc[:, 'Reviews']
reviews_dates = reviews.apply(lambda x: reviews_to_list(x, True))
#print(reviews_dates)
dates_df = pd.concat([pd.Series([date[0] for date in reviews_dates]),
                      pd.Series([date[1] for date in reviews_dates])], axis=1)

dates_df.columns = ['FRDate', 'SRDate']
dates_df

#dates_df['FRDate'] = pd.to_datetime(dates_df['FRDate'].dropna())
#dates_df['SRDate'] = pd.to_datetime(dates_df['SRDate'].dropna())

#dates_df.head()

,FRDate,SRDate
0,12/31/2017,11/20/2017
1,07/06/2017,06/19/2016
2,01/08/2018,01/06/2018
3,NaN,NaN
4,11/18/2017,02/19/2017
5,09/16/2017,05/31/2017
6,08/04/2017,11/05/2017
7,02/16/2014,06/13/2012
8,NaN,NaN
9,11/07/2017,03/27/2017


In [61]:
data.Reviews.to_csv('reviews.csv')

In [117]:
print(max(dates_df['FRDate'].dropna()), max(dates_df['SRDate'].dropna()))

ve tried"], [ 12/31/2017


In [53]:
dates_df['FRDate'].unique()

array(['12/31/2017', '07/06/2017', '01/08/2018', ..., 'and easy to u...',
       '08/10/2012', '05/31/2013'], dtype=object)

In [63]:
reviews_to_list("[['Fresh, hot, wonderful dumplings!'], ['03/19/2017']]", False)

['Fresh', 'hot']

In [84]:
"[['Fresh, hot, wonderful dumplings!'], ['03/19/2017']]"[1:-1]#.replace('[', '').replace(']', '').split('\', \'')

"['Fresh, hot, wonderful dumplings!'], ['03/19/2017']"

In [132]:
#(\'\d\d\/\d\d\/\d\d\d\d\')
#(\'[^\']+\')|(\"\".*\"\")
#(\'[^\']+\')|(""(?:[^"\\]++|\\.)*+"")|(\"\".*\"\")
pattern = re.compile('[0-9|/]+')

In [134]:
pattern.findall("[[""Don't miss it !""], ['01/02/2015']]")

['01/02/2015']